In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. Veriyi pkl dosyasından yükle
file_path = 'hazirlanmis_etkilesim.pkl'
data = pd.read_pickle(file_path)

# İlk birkaç satırı görüntüle
print("Veri Setindeki İlk 5 Satır:")
print(data.head())

print("\n")

Veri Setindeki İlk 5 Satır:
    uye_id  urun_id           eklenme_tarihi  puan   event_type
0        0    72176  2024-08-05 05:43:02.000     1         view
1        0    67714  2024-08-05 05:45:51.000     5     purchase
2        0    66060  2024-08-05 05:53:56.000     4  add_to_cart
3  1659156    60388  2024-08-05 06:01:07.000     1         view
4        0    41669  2024-08-05 06:04:45.000     5     purchase




In [2]:
data.columns = ["uye_id", "urun_id", "eklenme_tarihi", "puan","event_type"]
data["eklenme_tarihi"] = pd.to_datetime(data["eklenme_tarihi"])

# Eğitim ve Test veri setini oluşturma (zaman sırasına göre ayırma)
data = data.sort_values("eklenme_tarihi")
train_size = int(len(data) * 0.8)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

In [3]:
# Eğitim ve test veri setlerinin boyutlarını kontrol etme
print(f"Eğitim veri seti boyutu: {train_data.shape}")
print(f"Test veri seti boyutu: {test_data.shape}")


Eğitim veri seti boyutu: (149420, 5)
Test veri seti boyutu: (37355, 5)


In [4]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Kullanıcı-Ürün matrisini oluşturma
user_item_matrix = train_data.pivot_table(index="uye_id", columns="urun_id", values="puan", fill_value=0)

# Ürün benzerlik matrisi (Kosinüs Benzerliği)
item_similarity = cosine_similarity(user_item_matrix.T)
item_similarity_df = pd.DataFrame(item_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns)

# Tahmin yapma
def predict_item_based(user_id, item_id, user_item_matrix, item_similarity_df):
    if item_id not in item_similarity_df.index or user_id not in user_item_matrix.index:
        return np.nan  # Ürün veya kullanıcı eksikse NaN döner
    similar_items = item_similarity_df[item_id].sort_values(ascending=False)
    similar_items = similar_items.drop(item_id, errors='ignore')  # Kendini hariç tut
    user_ratings = user_item_matrix.loc[user_id]
    relevant_ratings = user_ratings[similar_items.index]
    predicted_score = np.dot(similar_items.values, relevant_ratings.values) / (similar_items.sum() + 1e-9)
    return predicted_score


In [5]:
# MNS (Mean Number of Suggestions) hesaplama fonksiyonu
def calculate_mns(test_data, user_item_matrix, item_similarity_df, top_n=5):
    total_suggestions = 0  # Toplam öneri sayısı
    user_count = 0  # Kullanıcı sayısı
    
    for user_id in test_data["uye_id"].unique():
        if user_id not in user_item_matrix.index:
            continue  # Kullanıcının eğitim setinde olmaması durumunda atla
        
        # Kullanıcı için tahmin edilen Top-N ürünleri hesapla
        user_ratings = user_item_matrix.loc[user_id]
        scores = np.dot(item_similarity_df.values, user_ratings.values)
        item_scores = pd.Series(scores, index=item_similarity_df.index)
        
        # Tahminleri sırala ve Top-N seç
        top_items = item_scores.sort_values(ascending=False).head(top_n)
        recommended_items = top_items.index.tolist()
        
        # Toplam öneri sayısını artır
        total_suggestions += len(recommended_items)
        user_count += 1
    
    # Ortalama öneri sayısını hesapla
    mns = total_suggestions / user_count if user_count > 0 else 0
    return mns

# MNS hesaplama
mns_score = calculate_mns(test_data, user_item_matrix, item_similarity_df)
print(f"Mean Number of Suggestions (MNS): {mns_score:.4f}")


Mean Number of Suggestions (MNS): 5.0000


In [6]:
# MDS (Mean Diversity Score) hesaplama fonksiyonu
def calculate_mds(test_data, user_item_matrix, item_similarity_df, top_n=5):
    diversity_scores = []  # Çeşitlilik skorlarını tutacak liste
    
    for user_id in test_data["uye_id"].unique():
        if user_id not in user_item_matrix.index:
            continue  # Kullanıcının eğitim setinde olmaması durumunda atla
        
        # Kullanıcı için tahmin edilen Top-N ürünleri hesapla
        user_ratings = user_item_matrix.loc[user_id]
        scores = np.dot(item_similarity_df.values, user_ratings.values)
        item_scores = pd.Series(scores, index=item_similarity_df.index)
        
        # Tahminleri sırala ve Top-N seç
        top_items = item_scores.sort_values(ascending=False).head(top_n)
        recommended_items = top_items.index.tolist()
        
        # Önerilen ürünlerin birbirine benzerlik skorlarını hesapla
        pairwise_diversity = []
        for i, item1 in enumerate(recommended_items):
            for j, item2 in enumerate(recommended_items):
                if i != j:
                    pairwise_diversity.append(1 - item_similarity_df.loc[item1, item2])
        
        # Çeşitlilik skorunun ortalamasını hesapla
        if pairwise_diversity:
            diversity_scores.append(np.mean(pairwise_diversity))
    
    # Ortalama Çeşitlilik Skoru
    mds = np.mean(diversity_scores) if diversity_scores else 0
    return mds

# MDS hesaplama
mds_score = calculate_mds(test_data, user_item_matrix, item_similarity_df)
print(f"Mean Diversity Score (MDS): {mds_score:.4f}")


Mean Diversity Score (MDS): 0.7560


In [7]:
# MPS (Mean Popularity Score) hesaplama fonksiyonu
def calculate_mps(test_data, train_data, user_item_matrix, item_similarity_df, top_n=5):
    # Eğitim setinden ürün popülerlik skorlarını hesapla
    item_popularity = train_data.groupby("urun_id").size()  # Her ürünün seçilme sıklığını al
    max_popularity = item_popularity.max()  # Normalizasyon için en yüksek popülerlik değeri
    
    popularity_scores = []  # Popülerlik skorlarını tutacak liste
    
    for user_id in test_data["uye_id"].unique():
        if user_id not in user_item_matrix.index:
            continue  # Kullanıcının eğitim setinde olmaması durumunda atla
        
        # Kullanıcı için tahmin edilen Top-N ürünleri hesapla
        user_ratings = user_item_matrix.loc[user_id]
        scores = np.dot(item_similarity_df.values, user_ratings.values)
        item_scores = pd.Series(scores, index=item_similarity_df.index)
        
        # Tahminleri sırala ve Top-N seç
        top_items = item_scores.sort_values(ascending=False).head(top_n)
        recommended_items = top_items.index.tolist()
        
        # Önerilen ürünlerin popülerlik skorlarını hesapla
        for item_id in recommended_items:
            popularity_score = item_popularity.get(item_id, 0) / max_popularity  # Normalizasyon
            popularity_scores.append(popularity_score)
    
    # Ortalama Popülerlik Skoru
    mps = np.mean(popularity_scores) if popularity_scores else 0
    return mps

# MPS hesaplama
mps_score = calculate_mps(test_data, train_data, user_item_matrix, item_similarity_df)
print(f"Mean Popularity Score (MPS): {mps_score:.4f}")


Mean Popularity Score (MPS): 0.2367


In [8]:
# CC (Coverage) hesaplama fonksiyonu
def calculate_cc(test_data, user_item_matrix, item_similarity_df, top_n=5):
    all_recommended_items = set()  # Önerilen tüm ürünlerin seti
    all_possible_items = set(user_item_matrix.columns)  # Eğitim setindeki tüm ürünler
    
    for user_id in test_data["uye_id"].unique():
        if user_id not in user_item_matrix.index:
            continue  # Kullanıcının eğitim setinde olmaması durumunda atla
        
        # Kullanıcı için tahmin edilen Top-N ürünleri hesapla
        user_ratings = user_item_matrix.loc[user_id]
        scores = np.dot(item_similarity_df.values, user_ratings.values)
        item_scores = pd.Series(scores, index=item_similarity_df.index)
        
        # Tahminleri sırala ve Top-N seç
        top_items = item_scores.sort_values(ascending=False).head(top_n)
        recommended_items = top_items.index.tolist()
        
        # Önerilen ürünleri sete ekle
        all_recommended_items.update(recommended_items)
    
    # Coverage oranını hesapla
    coverage = len(all_recommended_items) / len(all_possible_items) if len(all_possible_items) > 0 else 0
    return coverage

# CC hesaplama
cc_score = calculate_cc(test_data, user_item_matrix, item_similarity_df)
print(f"Coverage (CC) Score: {cc_score:.4f}")


Coverage (CC) Score: 0.2379


In [9]:
# MAP (Mean Average Precision) hesaplama fonksiyonu
def calculate_map(test_data, user_item_matrix, item_similarity_df, top_n=5):
    average_precisions = []  # Her kullanıcı için Average Precision (AP) değerlerini saklamak için liste
    
    for user_id in test_data["uye_id"].unique():
        if user_id not in user_item_matrix.index:
            continue  # Kullanıcının eğitim setinde olmaması durumunda atla
        
        # Test verisindeki kullanıcının gerçek beğendiği ürünler
        user_test_data = test_data[test_data["uye_id"] == user_id]
        relevant_items = user_test_data["urun_id"].tolist()
        
        # Kullanıcı için tahmin edilen Top-N ürünleri hesapla
        user_ratings = user_item_matrix.loc[user_id]
        scores = np.dot(item_similarity_df.values, user_ratings.values)
        item_scores = pd.Series(scores, index=item_similarity_df.index)
        
        # Tahminleri sırala ve Top-N seç
        top_items = item_scores.sort_values(ascending=False).head(top_n)
        recommended_items = top_items.index.tolist()
        
        # Average Precision hesapla
        hits = 0
        precision_at_k = []
        for k, item in enumerate(recommended_items, start=1):
            if item in relevant_items:
                hits += 1
                precision_at_k.append(hits / k)
        
        # AP değeri
        if precision_at_k:
            average_precisions.append(np.mean(precision_at_k))
        else:
            average_precisions.append(0)
    
    # MAP değerini hesapla
    map_score = np.mean(average_precisions) if average_precisions else 0
    return map_score

# MAP hesaplama
map_score = calculate_map(test_data, user_item_matrix, item_similarity_df)
print(f"Mean Average Precision (MAP): {map_score:.4f}")


Mean Average Precision (MAP): 0.4279


In [10]:
import pickle

# Tüm metrikleri tek bir sözlükte topla
item_based_results = {
    "MAP": map_score,
    "CC": cc_score,
    "MNS": mns_score,
    "MDS": mds_score,
    "MPS": mps_score
}

# Tek bir dosyaya kaydet
with open("item_based_sonuclari.pkl", "wb") as f:
    pickle.dump(item_based_results, f)
